In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
with open('intents.json', 'r') as file:
    data = json.load(file)

In [11]:
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']


for intent in data['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [14]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [15]:
#training data creation
train = []
output_array = [0] * len(classes)

#create bag of words
for document in documents:
    bagofwords = []
    word_patt = document[0]
    word_patt = [lemmatizer.lemmatize(word.lower()) for word in word_patt]
    for word in words:
        bagofwords.append(1) if word in word_patt else bagofwords.append(0)
        
output_row = list(output_array)
output_row[classes.index(document[1])] = 1
train.append([bagofwords, output_row])

In [16]:
#shuffle features
random.shuffle(train)

#numpy array
train = np.array(train)

X_train = list(train[:,0])
y_train = list(train[:,1])

<ipython-input-16-3faeb94a4d0a>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train)


In [17]:
# build model
model = Sequential()
model.add(Dense(128, input_shape = (len(X_train[0]),), activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation = "softmax"))

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])

results = model.fit(np.array(X_train), np.array(y_train), epochs = 200, batch_size = 10, verbose = 1)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.8959 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 2.3714 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 24ms/step - loss: 2.1847 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 28ms/step - loss: 2.1517 - accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 23ms/step - loss: 1.9338 - accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 22ms/step - loss: 1.8647 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 32ms/step - loss: 1.5044 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 20ms/step - loss: 1.5409 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 47ms/step - loss: 0.5942 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 25ms/step - loss: 0.8492 - accuracy: 1.0000
Epoch

In [18]:
model.save("chatbot_model.h5", results)